#### Installing Transformers and Importing Dependencies

In [ ]:
%pip install transformers
%pip install tf-keras

In [3]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

#### Loading Summarization Pipeline

In [4]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Get Blogpost

In [79]:
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

r = requests.get(URL)

In [80]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])

In [81]:
results

[<h1 class="story-title" style="text-align:left">Will The Game Stop with Gamestop Or Is This Just The Beginning?</h1>,
 <p class="paragraph">The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business.</p>,
 <p class="paragraph">The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers.</p>,
 <p class="paragraph">One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management

In [82]:
text = [result.text for result in results]

article = ' '.join(text)

#### Break post into smaller segments

In [83]:
article = article.replace('.', '.<eos>')
article = article.replace('?', '?<eos>')
article = article.replace('!', '!<eos>')
sentences = article.split('<eos>')

In [84]:
sentences[1]

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business.'

In [85]:
max_chunk = 500
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [86]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

#### Summarize Text

In [87]:
res = summarizer(chunks, max_length=80, min_length=30, do_sample=False)

In [88]:
res

[{'summary_text': ' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich'},
 {'summary_text': ' Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’. The market here is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing. The fundamentals no longer matter; and this was demonstrated only 5 years'},
 {'summary_text': ' Greenspans moral hazard of late 1994/1995 fundamentally changed the market in a new way . Extraordinary people who were never wrong in risk-adjusted terms and who understood markets were carried 

In [89]:
' '.join([summ['summary_text'] for summ in res])

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich  Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’. The market here is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing. The fundamentals no longer matter; and this was demonstrated only 5 years  Greenspans moral hazard of late 1994/1995 fundamentally changed the market in a new way . Extraordinary people who were never wrong in risk-adjusted terms and who understood markets were carried out of them horizontally by players with leverage . Financial 

In [90]:
summary = ' '.join([summ['summary_text'] for summ in res])

#### Save Summary

In [91]:
with open('blogsummary1.txt', 'w') as f:
    f.write(summary)